In [ ]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from config import Config
from sklearn.model_selection import train_test_split
import gc

In [ ]:

train = pd.read_csv('/opt/ml/input/data/train_data.csv', parse_dates=["Timestamp"])
test = pd.read_csv('/opt/ml/input/data/test_data.csv', parse_dates=["Timestamp"])

dat = pd.concat([train, test], axis = 0)
dat = dat.sort_values(by = ['userID', 'Timestamp'])

_train = dat[dat['answerCode'] >= 0]
_test = dat[dat['answerCode'] < 0]

_train['train_valid'] = 0
_train.loc[_train.drop_duplicates(subset='userID', keep = 'last').index, 'train_valid'] = -1
_valid = _train[_train['train_valid'] == -1]
_train = _train[_train['train_valid'] == 0]



In [4]:
def data_argument(train):
    _train = train.copy()
    _train.reset_index(drop = True, inplace= True)
    _train.loc[_train.drop_duplicates(subset='userID', keep = 'last').index, 'tem'] = -1
    _valid = _train[_train['tem'] == -1]
    _train = _train[_train['tem'] == 0]
    return _train, _valid 

In [5]:
_train_x, _valid = data_argument(_train)
arg_train = []
now = _train_x
for _ in range(10):
    now, now1 = data_argument(now)
    arg_train.append(pd.concat([now, now1], axis = 0))
test = pd.concat([_train, _test], axis = 0)
valid = pd.concat([_train_x, _valid], axis = 0)
train = pd.concat(arg_train)

In [10]:
test['userID'].value_counts()

730     1859
481     1846
1112    1776
394     1773
926     1772
        ... 
7358      13
7414      12
7390      12
7252      12
7441       8
Name: userID, Length: 7442, dtype: int64

In [11]:
_test

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
1035,3,A050133008,A050000133,-1,2020-10-26 13:13:57,5289
1706,4,A070146008,A070000146,-1,2020-12-27 02:47:54,9080
3023,13,A070111008,A070000111,-1,2020-12-27 04:35:09,9660
4283,17,A090064006,A090000064,-1,2020-10-30 05:48:37,2611
4670,26,A060135007,A060000135,-1,2020-10-23 11:44:18,1422
...,...,...,...,...,...,...
260052,7395,A040122005,A040000122,-1,2020-09-08 02:05:20,10615
260067,7404,A030111005,A030000111,-1,2020-10-13 09:49:18,7636
260082,7416,A050193004,A050000193,-1,2020-10-04 02:44:41,10402
260097,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402
